<a href="https://colab.research.google.com/github/mustafaqazi/GIAIC-Q3-Python/blob/main/secure_app.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
!pip install streamlit cryptography pyngrok --quiet

In [23]:
%%writefile secure_app.py

import streamlit as st
from cryptography.fernet import Fernet, InvalidToken
import base64
import hashlib

def generate_key(passkey):
    return base64.urlsafe_b64encode(hashlib.sha256(passkey.encode()).digest())

def encrypt_data(data, passkey):
    key = generate_key(passkey)
    f = Fernet(key)
    return f.encrypt(data.encode())

def decrypt_data(token, passkey):
    key = generate_key(passkey)
    f = Fernet(key)
    return f.decrypt(token).decode()

# Initialize session state
if "page" not in st.session_state:
    st.session_state.page = "login"
if "is_logged_in" not in st.session_state:
    st.session_state.is_logged_in = False
if "failed_attempts" not in st.session_state:
    st.session_state.failed_attempts = 0
if "encrypted_data" not in st.session_state:
    st.session_state.encrypted_data = None

def login_page():
    st.title("🔐 Secure Data System - Login")
    username = st.text_input("Username")
    password = st.text_input("Password", type="password")
    if st.button("Login"):
        if username == "admin" and password == "admin":
            st.session_state.is_logged_in = True
            st.session_state.failed_attempts = 0
            st.session_state.page = "main"
            st.success("✅ Login successful")
        else:
            st.error("❌ Invalid credentials")

def main_app():
    st.title("🛡️ Secure Data Encryption System")
    menu = ["Store Data", "Retrieve Data"]
    choice = st.radio("Choose an action", menu)

    if choice == "Store Data":
        data = st.text_area("Enter data to store securely")
        passkey = st.text_input("Set your passkey", type="password")
        if st.button("Encrypt & Store"):
            if data and passkey:
                encrypted = encrypt_data(data, passkey)
                st.session_state.encrypted_data = encrypted
                st.success("✅ Data encrypted and stored.")
                st.download_button("Download Encrypted Data", data=encrypted, file_name="data.enc", mime="application/octet-stream")
            else:
                st.warning("⚠️ Enter both data and passkey.")

    elif choice == "Retrieve Data":
        passkey = st.text_input("Enter passkey to decrypt", type="password")
        if st.button("Decrypt & Retrieve"):
            if st.session_state.encrypted_data is None:
                st.warning("⚠️ No data stored.")
            else:
                try:
                    decrypted = decrypt_data(st.session_state.encrypted_data, passkey)
                    st.success("✅ Decrypted Data:")
                    st.code(decrypted)
                    st.session_state.failed_attempts = 0
                except InvalidToken:
                    st.session_state.failed_attempts += 1
                    st.error("❌ Incorrect passkey!")
                    if st.session_state.failed_attempts >= 3:
                        st.warning("🔁 Too many failed attempts. Please login again.")
                        st.session_state.page = "login"
                        st.session_state.is_logged_in = False

# Routing
if st.session_state.page == "login" or not st.session_state.is_logged_in:
    login_page()
else:
    main_app()


Overwriting secure_app.py


In [24]:
from pyngrok import ngrok
import threading
import os
import time

# Optional: insert your authtoken if not already done
# from pyngrok import conf
# conf.get_default().auth_token = "YOUR_NGROK_TOKEN"

from pyngrok import conf
conf.get_default().auth_token = "2uAR9ADeEzrfxiM3G6dJL1rlppw_68kkWNC2pvj46pVYbJHAy"

# Kill old tunnels
ngrok.kill()

# Start Streamlit
def run():
    os.system("streamlit run secure_app.py")

threading.Thread(target=run).start()

time.sleep(4)  # wait for app to launch

public_url = ngrok.connect(8501)
print("🔗 Streamlit app is live at:", public_url)


🔗 Streamlit app is live at: NgrokTunnel: "https://aa7a-35-188-79-229.ngrok-free.app" -> "http://localhost:8501"
